In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import category_encoders as cs
from sklearn.pipeline import FeatureUnion
from sklearn.metrics import accuracy_score

In [2]:
worldcup=pd.read_csv("2018 worldcup.csv",index_col=0)
#match date is assumed to be irrelevant for the match results
worldcup.drop(['Date','Team1_Ball_Possession(%)'],axis=1,inplace=True)
#worldcup.describe()
train_wc = worldcup[:int(len(worldcup))]

In [3]:
#world cup attributes
w_features=train_wc.iloc[:,np.arange(26)].copy()
#world cup goal result
w_goals=train_wc.iloc[:,26].copy()
#wordl cup match result
w_results=train_wc.iloc[:,27].copy()

len(train_wc.columns)

28

In [4]:
# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames in this wise manner yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [5]:
def feature_process(features):

    w_features_num = features.drop(['Location','Phase','Team1','Team2','Team1_Continent','Team2_Continent','Normal_Time'], axis=1,inplace=False)
    w_features_cat= features[['Location','Phase','Team1','Team2','Team1_Continent','Team2_Continent','Normal_Time']].copy()


    num_pipeline = Pipeline([
            ('selector', DataFrameSelector(list(w_features_num))),
            ('imputer', Imputer(strategy="median")),
            ('std_scaler', StandardScaler(with_mean = False)),
        ])

    cat_pipeline = Pipeline([
            ('selector', DataFrameSelector(list(w_features_cat))),
            ('cat_encoder', cs.OneHotEncoder(drop_invariant=True)),
        ])

    full_pipeline = FeatureUnion(transformer_list=[
            ("num_pipeline", num_pipeline),
            ("cat_pipeline", cat_pipeline),
        ])
    return full_pipeline

In [6]:
full_pipeline = feature_process(w_features)

feature_prepared = pd.DataFrame(data=full_pipeline.fit_transform(w_features),index=np.arange(1,len(w_features) + 1))

worldcup_cleaned=pd.concat([feature_prepared,w_goals.to_frame(), w_results.to_frame()], axis=1)

In [7]:
# Linear Regression Model
from sklearn.linear_model import LinearRegression

train_data = feature_prepared[:int(0.8 * len(feature_prepared))]
test_data = feature_prepared[int(0.8 * len(feature_prepared)):]
train_goals = w_goals[:int(0.8 * len(w_goals))]
test_goals = w_goals[int(0.8 * len(w_goals)):]

model = LinearRegression(n_jobs = None)
model.fit(train_data, train_goals)
T_predict = model.predict(test_data)
W_predict = model.predict(train_data)
print model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)


In [8]:
from sklearn.metrics import mean_squared_error, r2_score


print(' ')
# The coefficients
#print('Coefficients and Intercept are: ', model.coef_,"   ",model.intercept_,' respectively')
# The mean squared error
print('_________________###################____________________')
print("Mean squared error for testing data: %.2f"
      % mean_squared_error(test_goals, T_predict))
# Explained variance score: 1 is perfect prediction
print('Variance score for testing data: %.2f' % r2_score(test_goals, T_predict))
print('******************************************************* ')
print("Mean squared error for training data: %.2f"
      % mean_squared_error(train_goals, W_predict))
# Explained variance score: 1 is perfect prediction
print('Variance score for training data: %.2f' % r2_score(train_goals, W_predict))

 
_________________###################____________________
Mean squared error for testing data: 5.29
Variance score for testing data: -1.57
******************************************************* 
Mean squared error for training data: 0.00
Variance score for training data: 1.00


In [9]:
# Linear Ridge Regression
from sklearn.linear_model import Ridge

model = Ridge(alpha=1.0)
model.fit(train_data, train_goals)
T_predict = model.predict(test_data)
W_predict = model.predict(train_data)
print model

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)


In [10]:
print(' ')
# The coefficients
#print('Coefficients and Intercept are: ', model.coef_,"   ",model.intercept_,' respectively')
# The mean squared error
print('_________________###################____________________')
print("Mean squared error for testing data: %.2f"
      % mean_squared_error(test_goals, T_predict))
# Explained variance score: 1 is perfect prediction
print('Variance score for testing data: %.2f' % r2_score(test_goals, T_predict))
print('******************************************************* ')
print("Mean squared error for training data: %.2f"
      % mean_squared_error(train_goals, W_predict))
# Explained variance score: 1 is perfect prediction
print('Variance score for training data: %.2f' % r2_score(train_goals, W_predict))

 
_________________###################____________________
Mean squared error for testing data: 3.98
Variance score for testing data: -0.93
******************************************************* 
Mean squared error for training data: 0.15
Variance score for training data: 0.94


In [16]:
# DesicionTree For Classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

train_data = feature_prepared[:int(0.8 * len(feature_prepared))]
test_data = feature_prepared[int(0.8 * len(feature_prepared)):]
train_results = w_results[:int(0.8 * len(w_results))]
test_results = w_results[int(0.8 * len(w_results)):]

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1)
grid_search_cv.fit(train_data, train_results)
print grid_search_cv.best_estimator_


T_predict = grid_search_cv.predict(test_data)
print("The prediction accuracy using the decision tree is : {:.2f}%.".format(100*accuracy_score(test_results, T_predict)))

# F1 score: 1 is perfect prediction
print("The F1 score using the decision tree is : {:.2f}.".format(f1_score(test_results,T_predict,average='weighted'))) 


fi1 = zip(w_features.columns, grid_search_cv.best_estimator_.feature_importances_)
fi1.sort(key = lambda x:-x[1])
pd.DataFrame(fi1, columns=["Feature","Importance"])

Fitting 3 folds for each of 294 candidates, totalling 882 fits
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=2, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best')
The prediction accuracy using the decision tree is : 53.85%.
The F1 score using the decision tree is : 0.45.


[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:    2.1s finished


,Feature,Importance
0,Location,1.0
1,Phase,0.0
2,Team1,0.0
3,Team1_Continent,0.0
4,Team1_Attempts,0.0
5,Team1_Corners,0.0
6,Team1_Offsides,0.0
7,Team1_Pass_Accuracy(%),0.0
8,Team1_Distance_Covered,0.0
9,Team1_Ball_Recovered,0.0


In [14]:
# Random Forest For Classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

train_data = feature_prepared[:int(0.8 * len(feature_prepared))]
test_data = feature_prepared[int(0.8 * len(feature_prepared)):]
train_results = w_results[:int(0.8 * len(w_results))]
test_results = w_results[int(0.8 * len(w_results)):]

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(RandomForestClassifier(random_state=42), params, n_jobs=-1, verbose=1)
grid_search_cv.fit(train_data, train_results)
print grid_search_cv.best_estimator_

T_predict = grid_search_cv.predict(test_data)
print("The prediction accuracy using the random forest is : {:.2f}%.".format(100*accuracy_score(test_results, T_predict)))

# F1 score: 1 is perfect prediction
print("The F1 score using the decision tree is : {:.2f}.".format(f1_score(test_results,T_predict,average='weighted'))) 


fi1 = zip(w_features.columns, grid_search_cv.best_estimator_.feature_importances_)
fi1.sort(key = lambda x:-x[1])
pd.DataFrame(fi1, columns=["Feature","Importance"])

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 744 tasks      | elapsed:   13.7s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=8,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
The prediction accuracy using the random forest is : 38.46%.
The F1 score using the decision tree is : 0.36.


[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:   16.1s finished


,Feature,Importance
0,Team1_Continent,0.148568
1,Location,0.102880
2,Team1_Ball_Recovered,0.072329
3,Team2_Ball_Possession(%),0.067682
4,Team2_Continent,0.047551
5,Team2_Corners,0.044898
6,Team1_Distance_Covered,0.040233
7,Team1_Fouls,0.037818
8,Team1_Red_Card,0.033207
9,Team2_Attempts,0.029985


In [17]:
# Naive Bayes For Classification
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(train_data, train_results)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

T_predict = model.predict(test_data)
print model
print("The prediction accuracy using the Navie Bayes is : {:.2f}%.".format(100*accuracy_score(test_results, T_predict)))

# F1 score: 1 is perfect prediction
print("The F1 score using the Naive Bayes is : {:.2f}.".format(f1_score(test_results,T_predict,average='weighted'))) 



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
The prediction accuracy using the Navie Bayes is : 61.54%.
The F1 score using the decision tree is : 0.51.


In [18]:
# Perceptron For Classification
from sklearn.linear_model import Perceptron

model = Perceptron(alpha=1)
model.fit(train_data, train_results)

T_predict = model.predict(test_data)
print model
print("The prediction accuracy using the Perceptron is : {:.2f}%.".format(100*accuracy_score(test_results, T_predict)))

# F1 score: 1 is perfect prediction
print("The F1 score using the Perceptron is : {:.2f}.".format(f1_score(test_results,T_predict,average='weighted'))) 



Perceptron(alpha=1, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False)
The prediction accuracy using the Perceptron is : 23.08%.
The F1 score using the decision tree is : 0.09.


/Users/zhuhualong/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [19]:
# Nearest Neighbour Classifer For Classification

from sklearn.neighbors import KNeighborsClassifier


params = {'leaf_size': list(range(10, 50)), 'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9]}
grid_search_cv = GridSearchCV(KNeighborsClassifier(), params, n_jobs=-1, verbose=1)
grid_search_cv.fit(train_data, train_results)
print grid_search_cv.best_estimator_

T_predict = grid_search_cv.predict(test_data)
print("The prediction accuracy using the Nearest Neighbour Classifer is : {:.2f}%.".format(100*accuracy_score(test_results, T_predict)))

# F1 score: 1 is perfect prediction
print("The F1 score using the Nearest Neighbour Classifer is : {:.2f}.".format(f1_score(test_results,T_predict,average='weighted'))) 



Fitting 3 folds for each of 320 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Done 398 tasks      | elapsed:    1.4s


KNeighborsClassifier(algorithm='auto', leaf_size=10, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')
The prediction accuracy using the Nearest Neighbour Classifer is : 15.38%.
The F1 score using the decision tree is : 0.08.


[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:    2.8s finished


In [20]:
# SVC For Classification

from sklearn.svm import SVC
from sklearn.model_selection import KFold

param_grid = [
        {'kernel': ['rbf'], 'C': [ 2**x for x in range(0,6) ]},
    ]
inner_cv = KFold(n_splits=3, shuffle=True, random_state=1)
grid_search = GridSearchCV(SVC(), param_grid, cv=inner_cv,  n_jobs=1, scoring='accuracy',verbose=3)
grid_search.fit(train_data, train_results)
clf=grid_search.best_estimator_


T_predict = grid_search.predict(test_data)
print grid_search
print("The prediction accuracy using the SVC is : {:.2f}%.".format(100*accuracy_score(test_results, T_predict)))

# F1 score: 1 is perfect prediction
print("The F1 score using the Nearest Neighbour Classifer is : {:.2f}.".format(f1_score(test_results,T_predict,average='weighted'))) 



Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] kernel=rbf, C=1 .................................................
[CV] ............ kernel=rbf, C=1, score=0.470588235294, total=   0.0s
[CV] kernel=rbf, C=1 .................................................
[CV] ............ kernel=rbf, C=1, score=0.294117647059, total=   0.0s
[CV] kernel=rbf, C=1 .................................................
[CV] ............ kernel=rbf, C=1, score=0.352941176471, total=   0.0s
[CV] kernel=rbf, C=2 .................................................
[CV] ............ kernel=rbf, C=2, score=0.352941176471, total=   0.0s
[CV] kernel=rbf, C=2 .................................................
[CV] ............ kernel=rbf, C=2, score=0.176470588235, total=   0.0s
[CV] kernel=rbf, C=2 .................................................
[CV] ............ kernel=rbf, C=2, score=0.294117647059, total=   0.0s
[CV] kernel=rbf, C=4 .................................................
[CV] ............

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.1s finished
